In [1]:
import csv
import math
import pandas as pd
from itertools import chain, combinations
from numpy import linspace

In [2]:
def euklides_measure(vec1, vec2):
    result = 0
    if len(vec1) != len(vec2):
        return 0
    for i in range(len(vec1)):
        result = result + (vec1[i]-vec2[i])**2

    result = math.sqrt(result)

    return result

In [3]:
def get_csv(x):

    df = pd.read_csv(x)
    leng = len(df.columns)
    return df, leng



In [4]:
def compare(test, which_line, agent, a_index, dec_atr):
    test = test_file.iloc[which_line]

    #zliczac do zmiennej liczbe wartosci atrybutu decyzyjnego -> podawana w zmiennej
    vec_of_vecs = []

    train_vec = [None] * (a_index - 1)
    test_vec = [None] * (a_index - 1)

    #tworzenie wektora odległości bezwzględnej
    for index, row in agent.iterrows():
        sim_vec = [-1 for x in range(len(dec_atr))]

        for i in range (len(train_vec)):
            train_vec[i] = row[agent.columns[i]]
            test_vec[i] = test[agent.columns[i]]

        emeasure = euklides_measure(train_vec,test_vec)


        for atr in dec_atr:
            if row[agent.columns[a_index-1]] == atr:
                sim_vec[dec_atr.index(atr)] = emeasure
                vec_of_vecs.append(sim_vec)
    # print(vec_of_vecs)
    return vec_of_vecs


In [5]:
def find_dec_atr(files):
    dec_atr = []
    for file in files:
        [agent, a_index] = get_csv(file)
        for index, row in agent.iterrows():
            dec = row[agent.columns[a_index-1]]
            if dec not in dec_atr:
                dec_atr.append(dec)

    return dec_atr

In [6]:
def make_vector(files, test_file, which_line, num_of_dec):
    all_vecs = []
    for file in files:
        [agent, a_index] = get_csv(file)
        all_vecs.append(compare(test_file, which_line, agent, a_index, num_of_dec))
    return all_vecs



In [7]:
def find_the_closest(k, list_v, dec_atr, agent_num):

    final_vec = [[[0 for x in range(len(dec_atr))] for x in range(k)] for x in range(agent_num)]
    help_vec = [[[] for x in range(len(dec_atr))]for x in range(agent_num)]

    # print(list_v)

    for j in range(len(list_v)):
        for l in range(len(list_v[j])):
            for m in range(len(list_v[j][l])):
                if list_v[j][l][m] != -1:
                    help_vec[j][m].append(list_v[j][l][m])
    # print(help_vec)
    for i in range(k):
        del_ixs = [[0 for x in range(len(dec_atr))] for x in range(agent_num)]
        for agent_n in range(len(help_vec)):
            for dec_n in range(len(help_vec[agent_n])):
                mini = min(help_vec[agent_n][dec_n])
                del_ixs[agent_n][dec_n] = help_vec[agent_n][dec_n].index(mini)
                final_vec[agent_n][i][dec_n] = mini

        # print('do usu ',del_ixs)
        # print('ost ', final_vec)

    # deletion:
    # sorting the inner list to avoid moving important index down
        for i in range(len(del_ixs)):
            for j in range(len(del_ixs[i])):
                if len(help_vec[i][j]) > 1:
                    help_vec[i][j].pop(del_ixs[i][j])

    correction_vec = [[0 for x in range(len(dec_atr))]for x in range(agent_num)]

    return final_vec, correction_vec

In [8]:
def unify_k_vecs_new(vec, agent_num, dec_num, correction):
    result_vec = [[0 for x in range(dec_num)] for x in range(agent_num)]
    for agent in range(len(vec)):
        for k in range(len(vec[agent])):
            for dec in range(len(vec[agent][k])):
                result_vec[agent][dec] += vec[agent][k][dec]

                if k == len(vec[agent])-1 and correction[agent][dec] != 0:
                    for additional in range(correction[agent][dec]):
                        result_vec[agent][dec] += vec[agent][k][dec]


    #dzielenie przez liczbe k
    for i in range(len(result_vec)):
        for j in range(len(result_vec[i])):
            result_vec[i][j] /= (len(vec[i]) + correction[i][j])

    return result_vec

In [9]:
def change_to_relative_decisions(vec):
    for i in range(len(vec[0])):
        temp_vec = [0 for x in range(len(vec))]

        for j in range(len(temp_vec)):
            temp_vec[j] = vec[j][i]

        sum = 0
        for j in range(len(temp_vec)):
            sum += temp_vec[j]

        for j in range(len(temp_vec)):
            temp_vec[j] /= sum

        for j in range(len(temp_vec)):
            vec[j][i] = temp_vec[j]

    return vec

In [10]:
def change_to_relative_agents(vec):

    for i in range(len(vec)):
        sum = 0
        for j in range(len(vec[i])):
            sum += vec[i][j]
        for j in range(len(vec[i])):
            if sum != 0:
                vec[i][j] = 1 - vec[i][j]/sum
            else:
                vec[i][j] = 0
        sum = 0
        for j in range(len(vec[i])):
            sum += vec[i][j]
        for j in range(len(vec[i])):
            if sum != 0:
                vec[i][j] = vec[i][j]/sum
            else:
                vec[i][j] = 0

    return vec


In [11]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [12]:
#q od 0 do 1 - wartosc procentowa, po osiagnieciu ktorej decyzja jest przeglosowana
def Shapley_Shubik(vec, q, dec_atr):

    q_primary = q
    sums = [0 for x in range(len(dec_atr))]
    # print(vec)
    #na początku szukamy list dla poszczególnych decyzji, zapisujemy ją (listę) jako temp_vec
    for i in range(len(dec_atr)):
        q = q_primary
        #temp_vec jest pomocniczy, aby obliczyc ostateczną siłę agenta: [dec1a1, dec1a2, dec1a3..]
        temp_vec = [0 for x in range(len(vec))]

        temp_sum = 0
        #uzupelnienie wektora
        for m in range(len(vec)):
            temp_vec[m] = vec[m][i]
            temp_sum += temp_vec[m]

        q *= temp_sum
        # print('dla wektora: ',temp_vec,' q: ',q)
        #wyciągamy pierszy element tabeli i później odkładamy go na górę (koniec)
        agent_forces = []
        for l in range(len(temp_vec)):

            x = temp_vec.pop(0)

            subsets = list(powerset(temp_vec))
            sum_to_Shapley = 0
            div = 0

            #liczenie wartości sił agentów dla poszczególnych decyzji
            for sub in subsets:
                if sub:
                    sum = 0
                    for s in sub:
                        sum += s
                    if sum >= q:
                        pass
                    elif sum + x >= q:
                        sum_to_Shapley += (math.factorial(len(sub)) * math.factorial((len(temp_vec)+1)-len(sub)-1))

                else:
                    if x >= q:
                        sum_to_Shapley += (math.factorial(len(sub)) * math.factorial((len(temp_vec)+1)-len(sub)-1))

            temp_vec.append(x)
            agent_forces.append(sum_to_Shapley/math.factorial(len(temp_vec)))
        sums[i] = agent_forces

            # liczenie sumy wartości sił jako siły agenta
    final_indexes = [0 for x in range(len(vec))]
    for i in range(len(sums)):
        for j in range(len(sums[i])):
            final_indexes[j] += sums[i][j]
    # print(final_indexes)
            # mnozenie wektorow przez siłę
    for i in range(len(vec)):
        for j in range(len(vec[i])):
            vec[i][j] *= final_indexes[i]
    # print(vec)
            # sumowanie przemnozonych wektorów
    final_vec = [0 for x in range(len(dec_atr))]

    for i in range(len(dec_atr)):
        for j in range(len(vec)):
            final_vec[i] += vec[j][i]

            #wybieranie decyzji
    max_val = max(final_vec)
    decision = [dec_atr[i] for i,j in enumerate(final_vec) if j==max_val]
    return decision

In [13]:
def Banzhaf_Coleman(vec, q, dec_atr):
    q_primary = q
    agent_forces = []
    for i in range(len(dec_atr)):
        q = q_primary
        # print(vec)
        temp_vec = [0 for x in range(len(vec))]

        temp_sum = 0
        for m in range(len(vec)):
            temp_vec[m] = vec[m][i]
            temp_sum += temp_vec[m]

        q *= temp_sum

        subsets = list(powerset(temp_vec))


        vic = [0 for x in range(len(vec))]
        for sub in subsets:
            sum = 0
            for j in range(len(sub)):

                sum += sub[j]
            if sum >= q:
                for j in range(len(temp_vec)):
                    if temp_vec[j] in sub:
                        vic[j] += 1
                    else:
                        vic[j] -= 1

        sum = 0
        for j in range(len(vic)):
            sum += vic[j]

        for j in range(len(vic)):
            vic[j] /= sum
        agent_forces.append(vic)

    final_indexes = [0 for x in range(len(vec))]
    for i in range(len(agent_forces)):
        for j in range(len(agent_forces[i])):
            final_indexes[j] += agent_forces[i][j]


    for i in range(len(vec)):
        for j in range(len(vec[i])):
            vec[i][j] *= final_indexes[i]

    final_vec = [0 for x in range(len(dec_atr))]

    for i in range(len(dec_atr)):
        for j in range(len(vec)):
            final_vec[i] += vec[j][i]


    #wybieranie decyzji
    max_val = max(final_vec)
    decision = [dec_atr[i] for i,j in enumerate(final_vec) if j==max_val]
    return decision

In [14]:
#gdy program sie ZAKONCZY: na koncu lini wykonan powinno byc <function TextIOWrapper.close()>
#tutaj zmiana w files agent3 -> agent7, dodac 'vehicle/agent5/d6.csv' i d7
#wrzucic plik vehicle_result.csv do folderu vehicle/agent5
#utworzyc plik vehicle_result.csv (prawym na folder .idea -> new -> file)

files = ['SatImg/agent9/d1.csv','SatImg/agent9/d2.csv','SatImg/agent9/d3.csv',
         'SatImg/agent9/d4.csv','SatImg/agent9/d5.csv','SatImg/agent9/d6.csv',
         'SatImg/agent9/d7.csv','SatImg/agent9/d8.csv','SatImg/agent9/d9.csv']

test_file, t_index = get_csv('SatImg/test_w.csv')
dec_atr = find_dec_atr(files)
ageent_num = len(files)

#parametry testu
header = ['Nazwa algorytmu','Dokładność','q','k','Srednia ilosc decyzji']
for i in range(len(dec_atr)):
    header.append(dec_atr[i])

header = ','.join(map(str,header))
f = open('SatImg_results_agent9.csv','a',encoding='UTF8')
f.write(header)

space = [0.8]

shp = ["\nShapley_Shubik","\nBanzhaf_Coleman"]

dec_stat_sum = [0 for x in range(len(dec_atr))]
dec_stat = [0 for x in range(len(dec_atr))]

k = 1 #9 do zbadania jeszcze
sb = 1
# for sb in range(2):
for q in range(len(space)):
    zgadza = 0
    length = 0
    for i in range(len(test_file)):
        # for i in range(1):

        line_to_compare = i

        list_v = make_vector(files, test_file, line_to_compare, dec_atr)

        k_closest, correction = find_the_closest(k, list_v, dec_atr, ageent_num)

        # print('cmp: ',test_file.iloc[line_to_compare])
        # print('list: ',list_v)

        # print("k_closest: ", k_closest)
        # print("correction ", correction)

        result = unify_k_vecs_new(k_closest, ageent_num, len(dec_atr), correction)


        relative = change_to_relative_agents(result)

        # print(relative)
        # dec = [1]
        # test_list = [[0.5],[0.3],[0.2]]

        if sb == 0:
            f_list = Shapley_Shubik(relative, space[q], dec_atr)
        else:
            f_list = Banzhaf_Coleman(relative, space[q], dec_atr)

        # print(f_list)

        # print(test_file.iloc[line_to_compare][t_index - 1], " <- poprawna | odgadnieta -> ", f_list)

        dec_stat_sum[dec_atr.index(test_file.iloc[line_to_compare][t_index - 1])] += 1

        if test_file.iloc[line_to_compare][t_index - 1] in f_list and len(f_list) == 1:
            dec_stat[dec_atr.index(test_file.iloc[line_to_compare][t_index - 1])] += 1

            zgadza += 1
        length += len(f_list)

    data = [shp[sb],zgadza/len(test_file),space[q],k,length/len(test_file)]
    for i in range(len(dec_stat)):
        if dec_stat_sum[i] != 0:
            data.append(dec_stat[i]/dec_stat_sum[i])

    data = ','.join(map(str,data))
    f.flush()
    f.write(data)

    print('skutecznosc: ',zgadza/len(test_file),' q ',space[q],' k ',k)
f.close
    # Dla k = 3 i q = 0.6
    # Skuteczność: 0.7283464566929134
    # Srednia ilosc podejmowanych decyzji:  1.0

    #Poprawione:
    # Gdy odległość jest równa, bierzemy wszystkie równe wartości (uwzględnione oczywiście również przy jednoczeniu wektoró)
    # Podjęte decyzje to lista, w wypadku niejednoznacznej decyzji (wszystkie wartości max wchodzą).
    # Naprawiona miara Euklidesowa! Dawniej - gdy 0 to przydzielałem 1e-21, teraz zostaje 0
    # Drobne zmiany optymalizacyjne - czas skrócony do 1.8(3) min [z 2,5min]
    # Indeks Banzhafa

    # Do dodania:

    # testowac k 1-9 co 1 
    # testowac q 0.3-0.8 co 0.1


skutecznosc:  0.886  q  0.8  k  1


<function TextIOWrapper.close()>